In [90]:
import data_structure as ds
from data_structure import Gender
import pandas as pd
import numpy as np

In [95]:
df = ds.search_all()
### 4o
df = df[(df['round_'] == 5) | (df['round_'] == 6)]

,name,gender,current_salary,deserved_salary,round_
0,James,Gender.Male,60.0,90.0,5
0,James,Gender.Male,55.0,75.0,5
0,James,Gender.Male,70.0,90.0,5
0,James,Gender.Male,55.0,85.0,5
0,James,Gender.Male,65.0,85.0,5
...,...,...,...,...,...
0,Harley,Gender.Neutral,65.0,85.0,6
0,Harley,Gender.Neutral,68.0,85.0,6
0,Harley,Gender.Neutral,65.0,85.0,6
0,Harley,Gender.Neutral,70.0,90.0,6


In [92]:
male_mask = df['gender'] == Gender.Male
df_male = df[male_mask]
df_female = df[df['gender'] == Gender.Female]
df_neutral = df[df['gender'] == Gender.Neutral]

In [93]:
deserved_group_df = pd.concat(
                    [df_male.value_counts('deserved_salary').sort_index(),
                      df_female.value_counts('deserved_salary').sort_index(),
                      df_neutral.value_counts('deserved_salary').sort_index()],
                        axis=1).reset_index()
deserved_group_df.replace(np.nan, 0, inplace=True)
deserved_group_df = deserved_group_df.apply(lambda x: x.astype(int), axis=1)
deserved_group_df.columns = ['salary',	'male', 'female', 'neutral']

current_group_df = pd.concat(
                    [df_male.value_counts('current_salary').sort_index(),
                     df_female.value_counts('current_salary').sort_index(),
                     df_neutral.value_counts('current_salary').sort_index()],
                        axis=1).reset_index()
current_group_df.replace(np.nan, 0, inplace=True)
current_group_df = current_group_df.apply(lambda x: x.astype(int), axis=1)
current_group_df.columns = ['salary',	'male', 'female', 'neutral']

deserved_group_df = deserved_group_df.sort_values('salary')
current_group_df = current_group_df.sort_values('salary')

display(current_group_df)
display(deserved_group_df)

deserved_group_df['salary_group'] = pd.cut(deserved_group_df['salary'],
                                            [0,80, 90, 100],
                                            labels=['<80', '80-90', '>90'])
current_group_df['salary_group'] = pd.cut(current_group_df['salary'],
                                            [0, 60, 70, 100],
                                            labels=['>60', '60-70', '>70'])

deserved_group_df = deserved_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')
current_group_df = current_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')

print('des')
display(deserved_group_df)

print('cur')
display(current_group_df)

,salary,male,female,neutral
28,50,0,0,1
21,54,0,1,0
0,55,17,33,14
1,56,1,0,0
22,56,0,2,0
2,57,3,0,0
4,58,1,0,0
3,58,2,6,2
23,59,0,2,0
5,60,70,101,62


,salary,male,female,neutral
13,70,0,1,1
0,75,16,35,16
1,78,1,0,0
2,80,37,55,33
3,82,1,4,1
4,85,234,253,207
14,87,0,2,0
5,89,1,0,0
6,90,117,98,137
7,92,1,1,2


des


C:\Users\pvest\AppData\Local\Temp\ipykernel_30780\2985621969.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  deserved_group_df = deserved_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')
C:\Users\pvest\AppData\Local\Temp\ipykernel_30780\2985621969.py:33: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  current_group_df = current_group_df.groupby('salary_group').sum().reset_index().drop(columns='salary')


,salary_group,male,female,neutral
0,<80,54,91,50
1,80-90,353,357,345
2,>90,86,46,95


cur


,salary_group,male,female,neutral
0,>60,94,146,79
1,60-70,311,305,323
2,>70,93,48,98


In [94]:
from scipy import stats

observed_deserved = deserved_group_df[['male', 'female', 'neutral']].to_numpy()

mf_obs = [observed_deserved[:,0], observed_deserved[:,1]], 'male-female'
mn_obs = [observed_deserved[:,0], observed_deserved[:,2]], 'male-neutral'
fn_obs = [observed_deserved[:,1], observed_deserved[:,2]], 'female-neutral'

for obs, descr in [mf_obs, mn_obs, fn_obs]:
    chi2_stat, p_val, dof, expected = stats.chi2_contingency(obs)
    print(f"Chi-square Test {descr}:\nChi2 Stat: {chi2_stat}, P-value: {p_val}, Degrees of Freedom: {dof}")
print('\n\n')

observed_current = current_group_df[['male', 'female', 'neutral']].to_numpy()
mf_des = [observed_current[:,0], observed_deserved[:,1]], 'male-female'
mn_des = [observed_current[:,0], observed_deserved[:,2]], 'male-neutral'
fn_des = [observed_current[:,1], observed_deserved[:,2]], 'female-neutral'

for obs, descr in [mf_des, mn_des, fn_des]:
    chi2_stat, p_val, dof, expected = stats.chi2_contingency(obs)
    print(f"Chi-square Test {descr}:\nChi2 Stat: {chi2_stat}, P-value: {p_val}, Degrees of Freedom: {dof}")


Chi-square Test male-female:
Chi2 Stat: 21.58413562805774, P-value: 2.0561959530217348e-05, Degrees of Freedom: 2
Chi-square Test male-neutral:
Chi2 Stat: 0.6839012342767208, P-value: 0.7103832846024519, Degrees of Freedom: 2
Chi-square Test female-neutral:
Chi2 Stat: 29.13970417311421, P-value: 4.703202051984531e-07, Degrees of Freedom: 2



Chi-square Test male-female:
Chi2 Stat: 19.092581046266364, P-value: 7.146587298389644e-05, Degrees of Freedom: 2
Chi-square Test male-neutral:
Chi2 Stat: 15.16413305689731, P-value: 0.0005095072231577228, Degrees of Freedom: 2
Chi-square Test female-neutral:
Chi2 Stat: 64.85296875603463, P-value: 8.267168074967479e-15, Degrees of Freedom: 2
